In [1]:
# Add this at the beginning of the notebook, before any other imports
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('.')))

In [2]:
import asyncio
from sqlalchemy import select
from db.database import AsyncSessionLocal, SessionLocal
from models.models import User
from passlib.context import CryptContext

In [3]:
# Password context for hashing - this is the FastAPI recommended way
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

async def create_user():
    async with AsyncSessionLocal() as session:
        # Create a new user
        new_user = User(
            username="test",
            email="test@example.com",
            password_hash=pwd_context.hash("123456")  # Using FastAPI's recommended hashing
        )
        
        # Add to session and commit
        session.add(new_user)
        await session.commit()
        await session.refresh(new_user)
        
        # Verify the user was created
        result = await session.execute(select(User).filter_by(username="test"))
        created_user = result.scalar_one()
        print(f"Created user: {created_user.username} (ID: {created_user.id})")
    
    return created_user

In [6]:
created_user = await create_user()

Created user: test (ID: 1)


In [7]:
async def get_user_by_id(id: int):
    async with AsyncSessionLocal() as session:
        result = await session.execute(select(User).filter_by(id=id))
        return result.scalar_one()

In [8]:
current_user = await get_user_by_id(1)

In [9]:
current_user.username

'test'

In [10]:
created_user.password_hash
pwd_context.verify("123456", created_user.password_hash)

True